In [222]:
import numpy as np
import sklearn
import pandas as pd
import warnings
%reload_ext autoreload
%autoreload 2
%matplotlib inline
warnings.filterwarnings('ignore')

In [223]:
# Input_Data = pd.read_csv('../data/TADPOLE_InputData.csv')

# # normalize the format of DATE
# Input_Data['EXAMDATE'] = pd.to_datetime(Input_Data['EXAMDATE'],errors='coerce')

# bl_time = ['Month']
# fixed_features = ['PTID_Key','EXAMDATE']
# selected_features = ['CDRSB','ADAS11','RAVLT_immediate','Hippocampus','WholeBrain','Entorhinal','MidTemp','APOE4','AGE']
# predicted_features = ['DX_bl','DX','ADAS13','Ventricles','MMSE']
# df_data = Input_Data[fixed_features + selected_features + predicted_features + bl_time]

# # Sorting data
# df_data = df_data.sort_values(by=['PTID_Key'])

# # Sorting the EXAMDATE because the EXAMDATE of one objective are disorderly
# df_data = df_data.groupby(['PTID_Key']).apply(lambda x: x.sort_values(['EXAMDATE'], ascending = True))
# df_data = df_data.reset_index(drop=True)

# # df_data = df_data.dropna(axis=0)

# # Check how many missing data and the type of data
# print(df_data.head())
# print('-'*50)
# print(df_data.isnull().sum())
# print('-'*50)
# print(df_data.info())

In [224]:
df_data = pd.read_csv('data/Input_pca_0.95thres.csv')
df_data.head()

,0,1,2,3,4,5,6,7,8,9,...,271,272,273,274,275,276,277,PTID_Key,EXAMDATE,DX
0,-0.914682,-0.601231,-1.025777,0.646176,1.011424,0.545620,0.413133,0.157729,0.240342,1.240933,...,0.881033,-0.162544,1.078744,1.362077,1.059333,0.267904,-0.928759,1.0,2010-12-10,2
1,-0.551814,-0.979159,-1.117663,1.076771,0.625453,0.594147,0.200665,-0.109799,0.167253,1.085187,...,1.949152,-0.949106,-0.750425,0.614402,1.703636,-0.747710,-0.966335,1.0,2011-04-07,2
2,-0.819792,-0.982411,-1.010481,1.050002,0.690168,0.794730,0.281609,-0.584068,0.522935,1.314278,...,0.429191,-0.094298,0.603369,0.183042,0.871928,0.794578,0.589846,1.0,2011-09-08,2
3,-1.336588,0.124982,0.265170,0.702093,-0.806993,0.915643,-0.873193,0.532319,-1.443681,0.450639,...,1.360018,0.069672,0.544021,-1.105401,0.759985,1.659128,-0.180586,2.0,2006-07-21,0
4,-1.562420,0.529299,0.027853,0.916200,-1.044956,1.685090,-0.881126,0.370308,-0.712720,-0.208872,...,0.207917,-0.230468,1.791909,-0.451569,1.189610,0.329250,-1.143252,2.0,2007-01-16,0


In [225]:
train_target = pd.read_csv('data/train_preprocessed.csv')
# train_target = train_target.reset_index(drop=True)
train_target.head()

,Date,PTID_Key,CN_Diag,MCI_Diag,AD_Diag,ADAS13,Ventricles_Norm,MMSE
0,2013-03-28,8,0.0,1.0,0.0,5.0,0.012128,30.0
1,2013-10-31,8,0.0,1.0,0.0,5.0,0.012128,30.0
2,2014-04-28,8,0.0,1.0,0.0,5.0,0.012128,30.0
3,2013-02-04,18,0.0,1.0,0.0,9.0,0.020526,30.0
4,2013-09-03,18,0.0,1.0,0.0,10.5,0.020526,29.5


In [226]:
val_target = pd.read_csv('data/val_preprocessed.csv')
val_target.head()

,Date,PTID_Key,CN_Diag,MCI_Diag,AD_Diag,ADAS13,Ventricles_Norm,MMSE
0,2013-02-13,5,1.0,0.0,0.0,16.0,0.019279,28.0
1,2013-08-14,5,0.5,0.5,0.0,19.5,0.019279,27.5
2,2014-02-20,5,0.0,1.0,0.0,23.0,0.019279,27.0
3,2015-03-05,5,0.0,0.0,1.0,29.0,0.019279,27.0
4,2016-04-28,5,0.0,0.0,1.0,26.0,0.019279,19.0


In [227]:
def prep_output(data,date_name):
    ID_train = np.unique(data.PTID_Key.values)
    for ID in ID_train:
        idx_list = data[data['PTID_Key']==ID].index.values
        for idx in idx_list:
            data.loc[idx,'blTime'] = df_data[df_data['PTID_Key']==ID]['EXAMDATE'].values[0]
#             data.loc[idx,'DX_bl'] = df_data[df_data['PTID_Key']==ID]['DX'].values[0]
    data[date_name] = pd.to_datetime(data[date_name], errors='coerce')
    data['blTime'] = pd.to_datetime(data['blTime'], errors='coerce')
    data['Month']=np.ceil((data[date_name]-data['blTime'])/np.timedelta64(1, 'M'))
    
    return data

In [228]:
def check_pred_step(data):
    ID_train = np.unique(data.PTID_Key.values)
    for ID in ID_train:
#         print(ID)
        last_visit = df_data[df_data['PTID_Key']==ID]['Month'].values.tolist()[-1]
#         print(df_data[df_data['PTID_Key']==ID]['Month'].values.tolist())
        future_visit_last_predict = data[data['PTID_Key']==ID]['Month'].values.tolist()[-1]
#         print(data[data['PTID_Key']==ID]['Month'].values.tolist())
#         print('-'*50)
    predict_step = []
    for ID in ID_train:
        #print(ID)
        last_visit = df_data[df_data['PTID_Key']==ID]['Month'].values.tolist()[-1]
        #print(last_visit)
        future_visit_last_predict = data[data['PTID_Key']==ID]['Month'].values.tolist()[-1]
        future_visit_time = (future_visit_last_predict - last_visit)//6
        #print(future_visit_time)
        #print('-'*50)
        predict_step.append(future_visit_time)

    df_predict_step = pd.DataFrame(predict_step)
    print(df_predict_step.describe())

In [229]:
train_new=prep_output(train_target,'Date')
df_data=prep_output(df_data,'EXAMDATE')
check_pred_step(train_new)

                0
count  475.000000
mean     4.934737
std      2.019408
min      0.000000
25%      3.500000
50%      5.000000
75%      7.000000
max      9.000000


In [230]:
val_new=prep_output(val_target,'Date')
check_pred_step(val_new)

                0
count  155.000000
mean     5.238710
std      2.061029
min      1.000000
25%      4.000000
50%      6.000000
75%      7.000000
max      9.000000


## preparing for training

In this part, first we normalize the time interval to be uniform as 6 months, and we use the nearest history visit data to fill in the training data and their corresponding label of the disease classification as well as the regression results for ADAS13, Ventricles and MMSE.

**Notice** For the time series data formation, we assume the time interval between two consecutive visit is **6 months**, and according to our problem, we try to predict the future **8 visits' results** based on current record. Here the future 8 visits comes as a hyper-parameters which we would tune by cross-validation.

**For the data missing, we also use the neareast value to fill in the NaN.

In [231]:

def delet_short_visit(df_data_re,train_target_re):
    # print("nrow of past visit:", df_data_re.shape[0], end = "\n\n")
    # print("nrow of future visit:", train_target_re.shape[0], end = "\n\n")

    time_interval = 6
    ID_train = np.unique(train_target_re.PTID_Key.values)
    for ID in ID_train:
        past_visit = df_data_re[df_data_re['PTID_Key']==ID]['Month'].values
        idx = df_data_re[df_data_re['PTID_Key']==ID].index.values

        # print(past_visit//time_interval)
        # print(idx)
        visit_time = 0
        index_delete = []
        index = 0
        month_record = past_visit//time_interval
        for record in month_record:
            if record == visit_time:
                visit_time += 1
            elif record < visit_time:
                # print(idx[index])
                index_delete.append(idx[index])
            elif record > visit_time:
                visit_time = record
                visit_time += 1
            index += 1

        df_data_re = df_data_re.drop(index_delete).reset_index(drop=True)

        future_visit = train_target_re[train_target_re['PTID_Key']==ID]['Month'].values
        idx = train_target_re[train_target_re['PTID_Key']==ID].index.values
        if future_visit[0]//time_interval <= visit_time-1:
            train_target_re.loc[(train_target_re['PTID_Key']==ID)
                                & (train_target_re['Month']//time_interval<=visit_time-1),
                                'Month'] = int(visit_time*time_interval)

        future_visit = train_target_re[train_target_re['PTID_Key']==ID]['Month'].values
        index_delete = []
        index = 0
        # print(future_visit//time_interval)
        for month_record in future_visit/time_interval:
            if month_record == visit_time:
                visit_time += 1
            elif month_record < visit_time:
                index_delete.append(idx[index])
            elif month_record > visit_time:
                visit_time = month_record
                visit_time += 1
            index += 1

        train_target_re = train_target_re.drop(index_delete).reset_index(drop=True)

    print("nrow of past visit:", df_data_re.shape[0], end = "\n\n")
    print("nrow of future visit:", train_target_re.shape[0], end = "\n\n")
    train_target_re.head(10)
    
    return {'output': train_target_re, 'input': df_data_re}

In [232]:
def impute_long_visit(df_data_re,train_target_re):
    time_interval = 6
    ID_train = np.unique(train_target_re.PTID_Key.values)
    for ID in ID_train:
        past_visit = df_data_re[df_data_re['PTID_Key']==ID]['Month'].values

        # print('past:', past_visit//time_interval)
        visit_time = 0
        month_record = past_visit//time_interval
        for record in month_record:
            if record > visit_time:
                add_num = record - visit_time
                past_data = df_data_re.loc[(df_data_re['PTID_Key']==ID) & (df_data_re['Month']//time_interval==visit_time-1), :]
                past_data_index = past_data.index.values
                past_data = pd.DataFrame(np.repeat(past_data.values,add_num,axis=0))
                past_data.columns = df_data_re.columns
                past_data['Month'] = [i for i in (visit_time + np.arange(0,add_num))*time_interval]
                past_data.index = [i for i in past_data_index+np.arange(0,1,1/(add_num+1))[1:]]
                df_data_re = df_data_re.append(past_data, ignore_index=False)
                df_data_re = df_data_re.sort_index().reset_index(drop=True)
                visit_time = record
            visit_time += 1

        future_visit = train_target_re[train_target_re['PTID_Key']==ID]['Month'].values
        # print('past:', past_visit//time_interval, 'future:', future_visit//time_interval)
        month_record = future_visit//time_interval

        if month_record[0] > visit_time:
            past_data = train_target_re.loc[(train_target_re['PTID_Key']==ID), :].iloc[0:1]
            past_data['Month'] = visit_time*time_interval
            past_data['CN_Diag'] = np.nan
            past_data['MCI_Diag'] = np.nan
            past_data['AD_Diag'] = np.nan
            past_data['ADAS13'] = np.nan
            past_data['Ventricles_Norm'] = np.nan
            past_data['MMSE'] = np.nan
            past_data.index = past_data.index.values-0.1
            train_target_re = train_target_re.append(past_data, ignore_index=False).sort_index().reset_index(drop=True)

        future_visit = train_target_re[train_target_re['PTID_Key']==ID]['Month'].values
        month_record = future_visit//time_interval
        for record in month_record:
            if record > visit_time:
                add_num = record - visit_time
                past_data = train_target_re.loc[(train_target_re['PTID_Key']==ID) & (train_target_re['Month']//time_interval==visit_time-1), :]
                past_data_index = past_data.index.values
                past_data = pd.DataFrame(np.repeat(past_data.values,add_num,axis=0))
                past_data.columns = train_target_re.columns
                past_data['Month'] = [i for i in (visit_time + np.arange(0,add_num))*time_interval]
                past_data['CN_Diag'] = np.nan
                past_data['MCI_Diag'] = np.nan
                past_data['AD_Diag'] = np.nan
                past_data['ADAS13'] = np.nan
                past_data['Ventricles_Norm'] = np.nan
                past_data['MMSE'] = np.nan
                past_data.index = [i for i in past_data_index+np.arange(0,1,1/(add_num+1))[1:]]
                train_target_re = train_target_re.append(past_data, ignore_index=False)
                train_target_re = train_target_re.sort_index().reset_index(drop=True)
                visit_time = record
            visit_time += 1



    print("nrow of past visit:", df_data_re.shape[0], end = "\n\n")
    print("nrow of future visit:", train_target_re.shape[0], end = "\n\n")
    train_target_re.head(10)
    
    return {'output': train_target_re, 'input': df_data_re}

In [233]:
# Do linear interpolation if there is data before and after for the same object, then do ffil and bfil
def linear_interp(data):
    ID_list = np.unique(data.PTID_Key.values)
    # Create an empty dataframe with all columns from data
    Input_new=pd.DataFrame(columns=data.columns)
#     print(ID_list)
    for ID in ID_list:
#         print(ID)
        df=data[data['PTID_Key']==ID]
        # interpolate only for numeric data
        df=df.interpolate()
        # ffill, bfill numeric data that can't be interpolate as well as categorical data
        df=df.fillna(method='ffill')
        df=df.fillna(method='bfill')
        Input_new=pd.concat([Input_new, df], ignore_index=True)

    print("Remaining missing values: ", Input_new.isnull().sum().sum() )
    print("Filled percentage: ", (1- Input_new.isnull().sum().sum()/data.isnull().sum().sum())*100,"%")
    return Input_new

In [234]:
train_target_re=delet_short_visit(df_data,train_new)['output']
df_train_re=delet_short_visit(df_data,train_new)['input']

train_target_re=impute_long_visit(df_train_re,train_target_re)['output']
df_train_re=impute_long_visit(df_train_re,train_target_re)['input']

nrow of past visit: 8379

nrow of future visit: 1515

nrow of past visit: 8379

nrow of future visit: 1515

nrow of past visit: 8433

nrow of future visit: 2435

nrow of past visit: 8433

nrow of future visit: 2435



In [235]:
val_target_re=delet_short_visit(df_data,val_new)['output']
df_val_re=delet_short_visit(df_data,val_new)['input']

val_target_re=impute_long_visit(df_val_re,val_target_re)['output']
df_val_re=impute_long_visit(df_val_re,val_target_re)['input']

nrow of past visit: 8608

nrow of future visit: 530

nrow of past visit: 8608

nrow of future visit: 530

nrow of past visit: 8635

nrow of future visit: 848

nrow of past visit: 8635

nrow of future visit: 848



In [236]:
# df_trian_re=linear_interp(df_train_re)
# df_val_re=linear_interp(df_val_re)
print(df_train_re.isnull().sum().sum())
print(df_val_re.isnull().sum().sum())
train_target_re=linear_interp(train_target_re)
val_target_re=linear_interp(val_target_re)

0
0
Remaining missing values:  0
Filled percentage:  100.0 %
Remaining missing values:  0
Filled percentage:  100.0 %


In [237]:
print(train_target_re.shape)
print(val_target_re.shape)

(2435, 10)
(848, 10)


In [238]:
df_train_re.head()

,0,1,2,3,4,5,6,7,8,9,...,273,274,275,276,277,PTID_Key,EXAMDATE,DX,blTime,Month
0,-0.914682,-0.601231,-1.02578,0.646176,1.01142,0.54562,0.413133,0.157729,0.240342,1.24093,...,1.07874,1.36208,1.05933,0.267904,-0.928759,1,2010-12-10 00:00:00,2,2010-12-10 00:00:00,0.0
1,-0.551814,-0.979159,-1.11766,1.07677,0.625453,0.594147,0.200665,-0.109799,0.167253,1.08519,...,-0.750425,0.614402,1.70364,-0.74771,-0.966335,1,2011-04-07 00:00:00,2,2010-12-10 00:00:00,4.0
2,-0.819792,-0.982411,-1.01048,1.05,0.690168,0.79473,0.281609,-0.584068,0.522935,1.31428,...,0.603369,0.183042,0.871928,0.794578,0.589846,1,2011-09-08 00:00:00,2,2010-12-10 00:00:00,9.0
3,-1.33659,0.124982,0.26517,0.702093,-0.806993,0.915643,-0.873193,0.532319,-1.44368,0.450639,...,0.544021,-1.1054,0.759985,1.65913,-0.180586,2,2006-07-21 00:00:00,0,2006-07-21 00:00:00,0.0
4,-1.56242,0.529299,0.027853,0.9162,-1.04496,1.68509,-0.881126,0.370308,-0.71272,-0.208872,...,1.79191,-0.451569,1.18961,0.32925,-1.14325,2,2007-01-16 00:00:00,0,2006-07-21 00:00:00,6.0


In [239]:
train_target_re.head()

,Date,PTID_Key,CN_Diag,MCI_Diag,AD_Diag,ADAS13,Ventricles_Norm,MMSE,blTime,Month
0,2013-03-28,8,0.0,1.0,0.0,5.0,0.012128,30.0,2011-10-27,18.0
1,2013-10-31,8,0.0,1.0,0.0,5.0,0.012128,30.0,2011-10-27,25.0
2,2014-04-28,8,0.0,1.0,0.0,5.0,0.012128,30.0,2011-10-27,31.0
3,2013-02-04,18,0.0,1.0,0.0,9.0,0.020526,30.0,2011-01-04,26.0
4,2013-09-03,18,0.0,1.0,0.0,10.5,0.020526,29.5,2011-01-04,32.0


In [240]:
df_train_re.to_csv('data/lstm_input_train.csv',index=False)
train_target_re.to_csv('data/lstm_output_train.csv',index=False)
df_val_re.to_csv('data/lstm_input_val.csv',index=False)
val_target_re.to_csv('data/lstm_output_val.csv',index=False)